In [2]:
from copy import deepcopy
from itertools import combinations
import pickle
import typing as tp
from zipfile import ZipFile
import datetime

from lightfm import LightFM
from lightfm.data import Dataset as LFMDataset
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn.preprocessing import normalize
from transliterate import translit

In [3]:
RANDOM_STATE = 42

In [4]:
df = pd.read_parquet("/home/aivanov/ml-zvuk/finetuning_transformer/airflow-dags-rc/wave/examples/models/player_starts_train.parquet")
df.head()

,date,user_id,item_id,watch_time,is_autorized
0,2023-07-21 19:04:50+03:00,user_12964323,video_1042531,51,0
1,2023-07-21 02:02:41+03:00,user_16517,video_1707159,31,0
2,2023-07-21 22:00:47+03:00,user_15057892,video_1989987,9,0
3,2023-07-21 19:09:43+03:00,user_2846972,video_1356486,-1,0
4,2023-07-21 11:06:58+03:00,user_20517034,video_1380654,11,0


In [ ]:
df['datetime'] = pd.to_datetime(df['date']).dt.date

In [ ]:
videos = pd.read_parquet("/home/aivanov/ml-zvuk/finetuning_transformer/airflow-dags-rc/wave/examples/models/videos.parquet")
videos.head()

,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
0,video_165654,MSI Pro MP241X недоОБЗОР (РЕШЕНИЕ ПРОБЛЕМЫ С М...,Silvi,None,0,В видео я обывательским взглядом расскажу про ...,Технологии и интернет,2022-12-08 13:53:05+03:00,391382,0,0,NaN,0.0,0.0,NaN
1,video_1173704,Наложение пястно фаланговой повязки на кисть,"УЦ ""Академия Безопасности""",None,0,Видео с канала УЦ Академия безопасности (ab-dp...,Образование,2022-03-24 09:19:15+03:00,125922,26,0,NaN,0.0,0.0,0.0
2,video_23927,SilverstoneF1 Sochi Pro и Neoline x cop 6000s ...,Artur48,None,0,SilverstoneF1 Sochi Pro и Neoline x cop 6000s ...,Авто-мото,2022-03-19 17:41:49+03:00,436570,2,0,NaN,NaN,0.0,0.0
3,video_1003780,БОЛЬНИЦА в Brookhaven! ДОКТОР ПУПКИН спас ЖЕНИ...,ПАПА ДОЧКИ Games,None,0,Играем в Роблокс (Roblox) - БОЛЬНИЦА в Brookha...,Детям,2021-02-20 11:50:53+03:00,719377,673,0,0.0,1.0,0.0,0.0
4,video_105383,"Вебинар ""Особенности трудоустройства граждан Б...","ЗАО ""Сплайн-Центр""",None,0,"10.08.2023 Вебинар ""Особенности трудоустройств...",Бизнес и предпринимательство,2023-08-11 09:02:07+03:00,3834404,19,0,0.0,0.0,0.0,0.0


In [ ]:
train_start_date = datetime.date(2023, 7, 21)
train_end_date = datetime.date(2023, 8, 20)
validation_date = datetime.date(2023, 8, 21)

train = df.loc[
    (df["datetime"] >= train_start_date)
    & (df["datetime"] <= train_end_date),
    ["user_id", "item_id"]
]

test = df.loc[df["datetime"] == validation_date, ["user_id", "item_id"]]
test = test.loc[test["user_id"].isin(train["user_id"]) & test["item_id"].isin(train["item_id"])]

In [ ]:
train = pd.merge(train, videos[["item_id", "video_title", "category_title"]], on="item_id")
test = pd.merge(test, videos[["item_id", "video_title", "category_title"]], on="item_id")

In [ ]:
train.shape, test.shape

((67514739, 4), (1087375, 4))

In [40]:
lfm_dataset = LFMDataset()
lfm_dataset.fit(
    users=train["user_id"].values,
    items=train["item_id"].values,
)

train_matrix, _ = lfm_dataset.build_interactions(zip(*train[["user_id", "item_id"]].values.T))

In [41]:
lfm_model = LightFM(
    learning_rate=0.01, 
    loss='warp', 
    no_components=64,
    random_state=RANDOM_STATE
)

In [42]:
lfm_model.fit(
    interactions=train_matrix, 
    epochs=20,
    num_threads=20
);

In [43]:
n_recommendations = 10

In [44]:
id_item_mapping = {v: k for k, v in lfm_dataset._item_id_mapping.items()}

In [45]:
def get_n_recommendations_for_user(
    user_id: str,
    model: LightFM,
    train_matrix: coo_matrix,
    user_to_id: tp.Dict[str, int],
    id_to_item: tp.Dict[int, str],
    n_recommendations: int
) -> pd.DataFrame:
    user_inner_id = user_to_id[user_id]
    scores = model.predict(
        user_ids=user_inner_id,
        item_ids=np.arange(train_matrix.shape[1]),
        num_threads=60
    )
    user_watched_items = train_matrix.col[train_matrix.row == user_inner_id]
    scores[user_watched_items] = -np.inf

    recommended_item_inner_ids = np.argpartition(scores, -np.arange(n_recommendations))[
        -n_recommendations:
    ][::-1]
    recommended_item_ids = [id_to_item[x] for x in recommended_item_inner_ids]
    return recommended_item_ids

In [46]:
user_id = "user_12964323"

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations
)
recs_for_user_1 = pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(videos[["item_id", "video_title", "category_title"]])
recs_for_user_1

,user_id,item_id,video_title,category_title
0,user_12964323,video_170129,Филипп Янковский высказался об Андрее Тарковском,Искусство
1,user_12964323,video_1508131,Анастасия Попова («Бэби-тур») высказалась об о...,Лайфстайл
2,user_12964323,video_1989987,Юлия Пересильд о дружбе с мужчинами,Лайфстайл
3,user_12964323,video_1486704,Смешные животные ? / №1,Юмор
4,user_12964323,video_1448534,"Виктория Райдос о том, что «Битва экстрасенсо...",Лайфстайл
5,user_12964323,video_144691,"Выжить в Дубае, 5 выпуск",Телепередачи
6,user_12964323,video_803844,Прямой эфир Звезда,Телепередачи
7,user_12964323,video_302657,"Выжить в Дубае, 6 выпуск",Телепередачи
8,user_12964323,video_1738263,Прямой эфир Ю-ТВ,Телепередачи
9,user_12964323,video_291331,Из-за чего на самом деле сорвалась свадьба Еле...,Разное


In [47]:
user_id = "user_20385130"

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations
)
recs_for_user_2 = pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(videos[["item_id", "video_title", "category_title"]])
recs_for_user_2

,user_id,item_id,video_title,category_title
0,user_20385130,video_170129,Филипп Янковский высказался об Андрее Тарковском,Искусство
1,user_20385130,video_1508131,Анастасия Попова («Бэби-тур») высказалась об о...,Лайфстайл
2,user_20385130,video_1486704,Смешные животные ? / №1,Юмор
3,user_20385130,video_291331,Из-за чего на самом деле сорвалась свадьба Еле...,Разное
4,user_20385130,video_230144,Поместится третий: Тимати готовится к рождению...,Развлечения
5,user_20385130,video_100137,Канье Уэст пытается опротестовать брачный конт...,Разное
6,user_20385130,video_1989987,Юлия Пересильд о дружбе с мужчинами,Лайфстайл
7,user_20385130,video_1009706,Марина Александрова раскрыла секреты воспитани...,Развлечения
8,user_20385130,video_982310,Бывший муж Бородиной впервые вышел на связь по...,Развлечения
9,user_20385130,video_568418,Все-таки не одиноки? Новые свидетельства об НЛ...,Развлечения


In [48]:
# самые просматриваемые в обучающей выборке
train["video_title"].value_counts().head(10)

Филипп Янковский высказался об Андрее Тарковском                  720109
Анастасия Попова («Бэби-тур») высказалась об обнаженных сценах    610810
Выжить в Дубае, 5 выпуск                                          589184
Выжить в Дубае, 6 выпуск                                          552822
Выжить в Дубае, 7 выпуск                                          521438
Соловьёв LIVE | Круглосуточный канал                              512028
Прямой эфир Звезда                                                422066
Выжить в Дубае, 8 выпуск                                          410565
Прямой эфир ТНТ                                                   398527
Выжить в Дубае, 4 выпуск                                          332536
Name: video_title, dtype: int64

# Нормирование данных

In [49]:
lfm_model_cos = deepcopy(lfm_model)

lfm_model_cos.item_biases = np.zeros_like(lfm_model_cos.item_biases)
lfm_model_cos.user_biases = np.zeros_like(lfm_model_cos.user_biases)

lfm_model_cos.item_embeddings = normalize(lfm_model_cos.item_embeddings)
lfm_model_cos.user_embeddings = normalize(lfm_model_cos.user_embeddings)

In [50]:
user_id = "user_12964323"

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model_cos,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations
)
pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(videos[["item_id", "video_title", "category_title"]])

,user_id,item_id,video_title,category_title
0,user_12964323,video_2006802,Супер мощный светодиодный фонарик XLM-P70,Развлечения
1,user_12964323,video_917632,Как вам такая идея абстрактного дизайна ногтей?,Красота
2,user_12964323,video_538200,Работа на автомойке | Мои деньги #12,Лайфстайл
3,user_12964323,video_1433323,День защиты детей,Образование
4,user_12964323,video_805725,Крем для рук продлевает их молодость.MOV,Красота
5,user_12964323,video_376641,Crataegus -Q হার্টের রোগীদের জন্য মহাউপকারী এক...,Разное
6,user_12964323,video_1965063,Открытие МФЦ 08.06.2023 г. Ковернино.,Разное
7,user_12964323,video_765182,Суп с рыбными консервами и рисом. Рецепт без з...,Разное
8,user_12964323,video_1190387,Ловля ночного судака на воблеры. Часть 1,Охота и рыбалка
9,user_12964323,video_188033,СЕЗОН 4. Утренняя зарядка на все группы мышц |...,Спорт


In [51]:
user_id = "user_20385130"

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model_cos,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations
)
pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(videos[["item_id", "video_title", "category_title"]])

,user_id,item_id,video_title,category_title
0,user_20385130,video_1425143,Suzuki grand vitara xl-7 2.7 замена грм часть ...,Авто-мото
1,user_20385130,video_141133,ГТА 5 МОДЫ РЕАЛЬНАЯ ЖИЗНЬ БОЙ ПРОТИВ КОНОР МАК...,Видеоигры
2,user_20385130,video_1611265,Домашняя Пицца - Простой Рецепт от Бабушки Эммы,Кулинария
3,user_20385130,video_1021787,Догонялки,Животные
4,user_20385130,video_1708464,Подопытные Космонавты Пережившие Ад _ Барокаме...,Образование
5,user_20385130,video_2308357,Как побороть страх,Эзотерика
6,user_20385130,video_86224,смена масла в кпп на хонде аккорд.mp4,Авто-мото
7,user_20385130,video_1052196,ПОППИ ПЛЕЙТАЙМ vs МАЙНКРАФТ,Видеоигры
8,user_20385130,video_1935727,Сборка фрамуги c фурнитурой VORNE на профиле L...,Строительство и ремонт
9,user_20385130,video_1210234,"Трейлер ""СЕРДИТАЯ ЧЕРНАЯ ДЕВУШКА И ЕЕ МОНСТР"" ...",Фильмы


In [52]:
models_dict = {"lfm": lfm_model, "lfm_cos": lfm_model_cos}

In [53]:
model_name = "lfm_cos"
model = lfm_model_cos

In [56]:
sample_submission = pd.read_csv("/home/aivanov/ml-zvuk/finetuning_transformer/airflow-dags-rc/wave/examples/models/sample_submission.csv")
sample_submission

,user_id,recs
0,user_26511551,"['video_0', 'video_0', 'video_0', 'video_0', '..."
1,user_29194819,"['video_0', 'video_0', 'video_0', 'video_0', '..."
2,user_29734049,"['video_0', 'video_0', 'video_0', 'video_0', '..."
3,user_955460,"['video_0', 'video_0', 'video_0', 'video_0', '..."
4,user_7065521,"['video_0', 'video_0', 'video_0', 'video_0', '..."
...,...,...
97235,user_29281681,"['video_0', 'video_0', 'video_0', 'video_0', '..."
97236,user_3912848,"['video_0', 'video_0', 'video_0', 'video_0', '..."
97237,user_28389099,"['video_0', 'video_0', 'video_0', 'video_0', '..."
97238,user_18951296,"['video_0', 'video_0', 'video_0', 'video_0', '..."


In [58]:
del sample_submission['recs']

In [60]:
recommendations = sample_submission

In [78]:
recommendations = df[df.user_id.isin(sample_submission.user_id.values)].user_id.unique()

In [80]:
len(recommendations)

57822

In [81]:
recommendations = pd.DataFrame({'user_id': recommendations})

In [83]:
recommendations["item_id"] = recommendations["user_id"].apply(
    get_n_recommendations_for_user,
    args=(
        model,
        train_matrix,
        lfm_dataset._user_id_mapping,
        id_item_mapping,
        n_recommendations
    ),
)
recommendations = recommendations.explode("item_id")
recommendations["rank"] = recommendations.groupby(["user_id"]).cumcount() + 1

In [33]:
videos[["item_id", "category_title"]].head()

,item_id,category_title
0,video_165654,Технологии и интернет
1,video_1173704,Образование
2,video_23927,Авто-мото
3,video_1003780,Детям
4,video_105383,Бизнес и предпринимательство


In [36]:
videos.duplicated().any()

True

In [37]:
item_genres_one_hot = videos[["item_id", "category_title"]].copy()
item_genres_one_hot["category_title"] = item_genres_one_hot["category_title"].str.split(", ")
item_genres_one_hot = item_genres_one_hot.explode("category_title")
item_genres_one_hot["category_title"] = item_genres_one_hot["category_title"].str.replace(" ", "_")
item_genres_one_hot["category_title"] = item_genres_one_hot["category_title"].map(lambda x: translit(x, "ru", reversed=True))
item_genres_one_hot["value"] = 1
item_genres_one_hot = item_genres_one_hot.drop_duplicates()
item_genres_one_hot = item_genres_one_hot.pivot(
    index="item_id", 
    columns="category_title", 
    values="value"
).fillna(0).astype(int)

item_genres_one_hot.head()

category_title,Anime,Audio,Audioknigi,Avto-moto,Biznes_i_predprinimatel'stvo,Blogi,Detjam,Dizajn,Ezoterika,Fil'my,...,Sport,Sport/Igry,Stroitel'stvo_i_remont,Tehnika_i_oborudovanie,Tehnologii_i_internet,Teleperedachi,VEF,Videoigry,Zdorov'e,Zhivotnye
item_id,,,,,,,,,,,,,,,,,,,,,
video_0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
video_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
video_10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
video_100,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
video_1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
def get_hamming_distances(pairs: pd.Series, features: pd.DataFrame) -> np.ndarray:
    items_0 = pairs.map(lambda pair: pair[1]).values
    items_1 = pairs.map(lambda pair: pair[0]).values

    features_0 = features.reindex(items_0).values
    features_1 = features.reindex(items_1).values
    return np.sum(features_0 != features_1, axis=1)


def calculate_intra_list_diversity_per_user(recommendations: pd.DataFrame, features: pd.DataFrame) -> pd.Series:
    recommended_item_pairs = recommendations.groupby("user_id")["item_id"].apply(
        lambda x: list(combinations(x, 2))
    ).reset_index().explode("item_id").rename(columns={"item_id": "item_pair"})
    recommended_item_pairs["dist"] = get_hamming_distances(recommended_item_pairs["item_pair"], features)
    return recommended_item_pairs[["user_id", "dist"]].groupby("user_id").agg("mean")

In [39]:
for model_name, recommendations in recommendations_dict.items():
    ild_per_user = calculate_intra_list_diversity_per_user(recommendations, item_genres_one_hot)
    print(f"model: {model_name}, mean ild: {round(float(ild_per_user.mean()), 2)}\n")

model: lfm, mean ild: 0.75

model: lfm_cos, mean ild: 1.17



In [ ]:
def compute_metrics(df_true, df_pred, top_N):
    result = {}
    test_recs = df_true.set_index(['user_id', 'item_id']).join(df_pred.set_index(['user_id', 'item_id']))
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])

    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']
    
    users_count = test_recs.index.get_level_values('user_id').nunique()
    for k in range(1, top_N + 1):
        hit_k = f'hit@{k}'
        test_recs[hit_k] = test_recs['rank'] <= k
        result[f'Precision@{k}'] = (test_recs[hit_k] / k).sum() / users_count
        result[f'Recall@{k}'] = (test_recs[hit_k] / test_recs['users_item_count']).sum() / users_count

    result[f'MAP@{top_N}'] = (test_recs["cumulative_rank"] / test_recs["users_item_count"]).sum() / users_count
    result[f'MRR'] = test_recs.groupby(level='user_id')['reciprocal_rank'].max().mean()
    return pd.Series(result)